# Text Detection using Amazon Rekognition

***
This notebook provides a walkthrough of [object detection API](https://docs.aws.amazon.com/rekognition/latest/dg/labels.html) in Amazon Rekognition. You can quickly identify objects in your video and image libraries to catalog footage and photos for marketing, advertising, and media industry use cases.
***

# Initialize stuff

In [39]:
# Initialise Notebook
import boto3
from IPython.display import HTML, display, Image as IImage
from PIL import Image, ImageDraw, ImageFont
import time
import os
from trp import Document

In [40]:
# Curent AWS Region. Use this to choose corresponding S3 bucket with sample content

mySession = boto3.session.Session()
awsRegion = mySession.region_name

In [41]:
# Init clients
textract = boto3.client('textract')
s3 = boto3.client('s3')

In [42]:
# S3 bucket that contains sample images and videos

# We are providing sample images and videos in this bucket so
# you do not have to manually download/upload test images and videos.

bucketName = "aws-workshops-" + awsRegion

In [43]:
# Create temporary directory
# This directory is not needed to call Rekognition APIs.
# We will only use this directory to download images from S3 bucket and drwaw bounding boxes
# around recognized celebrities to show them here in the notebook.

!mkdir m1tmp
tempFolder = 'm1tmp/'

mkdir: cannot create directory ‘m1tmp’: File exists


# Detect text in image
***

In [44]:
imageName = "content-moderation/media/simple-document-image.jpg"

In [45]:
display(IImage(url=s3.generate_presigned_url('get_object', Params={'Bucket': bucketName, 'Key': imageName})))

#### Call Rekognition to detect text in the image

In [46]:
# Call Amazon Textract
response = textract.detect_document_text(
    Document={
        'S3Object': {
            'Bucket': bucketName,
            'Name': imageName
        }
    })

#### Review the raw JSON reponse from Rekognition

In [47]:
# Show JSON response returned by Rekognition Labels API (Object Detection)
# In the JSON response below, you will see Label, detected instances, confidence score and additional information.

display(response)

{'DocumentMetadata': {'Pages': 1},
 'Blocks': [{'BlockType': 'PAGE',
   'Geometry': {'BoundingBox': {'Width': 1.0,
     'Height': 1.0,
     'Left': 0.0,
     'Top': 0.0},
    'Polygon': [{'X': 0.0, 'Y': 0.0},
     {'X': 1.0, 'Y': 0.0},
     {'X': 1.0, 'Y': 1.0},
     {'X': 0.0, 'Y': 1.0}]},
   'Id': 'dd0f580a-76d1-434f-a9e2-c63962dc4e1d',
   'Relationships': [{'Type': 'CHILD',
     'Ids': ['dc4ad1c9-b37b-417a-903f-41798c46619c',
      '8adabc59-fa1c-4daa-b018-111e3bd3f586',
      '4e5a4790-d993-4916-96f0-1080280e4a83',
      '9c998aa6-904f-487e-8861-b567c6f8f984']}]},
  {'BlockType': 'LINE',
   'Confidence': 98.11465454101562,
   'Text': 'Amazon.com, Inc. is located in Seattle, WA',
   'Geometry': {'BoundingBox': {'Width': 0.512601375579834,
     'Height': 0.06267166882753372,
     'Left': 0.06366883218288422,
     'Top': 0.20170557498931885},
    'Polygon': [{'X': 0.06366883218288422, 'Y': 0.20170557498931885},
     {'X': 0.5762702226638794, 'Y': 0.20170557498931885},
     {'X': 0.576

#### Display detected lines

In [48]:

# Print detected text
for item in response["Blocks"]:
    if item["BlockType"] == "LINE":
        print ('\033[94m' +  item["Text"] + '\033[0m')

Amazon.com, Inc. is located in Seattle, WA
It was founded July 5th, 1994 by Jeff Bezos
Amazon.com allows customers to buy everything from books to blenders
Seattle is north of Portland and south of Vancouver, BC.


In [49]:
unsafeWords = ["WA"]
for item in response["Blocks"]:
    if item["BlockType"] == "WORD":
        if(item["Text"] in unsafeWords):
            print("Detected unsafe word: {}".format(item["Text"]))

Detected unsafe word: WA


***
### References
- https://docs.aws.amazon.com/rekognition/latest/dg/API_DetectText.html

***

You have successfully used Amazon Rekognition to identify celebrities in images an videos. In the next module, Recognize Custom Celebrities, you will learn how to recognize your custom celebrities in the images and videos.